In [130]:
import numpy as np
from collections import defaultdict
from itertools import combinations
import time

class Dataset_Sequence:
    """Utility class to manage a dataset stored in a external file."""

    def __init__(self, filepath):
        """reads the dataset file and initializes files"""
        """This class reads the file for sequence mining. Each line is an item 
        with its location and the transactions are separated by '\n'"""
        self.transactions = list()
        self.items = set()
        self.Freq_Items={}
        #self.minSup=minSup
        self.Vertical_Rep={}
        try:
            f=open(filepath,'r')
            line=[]
            idx=0
            Freq_Items={}
            for itemLoc in f.readlines():
                if itemLoc!='\n':
                    itemLoc=itemLoc.strip()
                    item=itemLoc.split(' ')[0]
                    loc=int(itemLoc.split(' ')[1])
                    line.append(item)
                    if tuple([item]) not in self.Vertical_Rep:
                        self.Vertical_Rep[tuple([item])]={}
                    if idx not in self.Vertical_Rep[tuple([item])]:
                        self.Vertical_Rep[tuple([item])][idx]=[]
                    
                    self.Vertical_Rep[tuple([item])][idx].append(loc)
                    #self.Vertical_Rep[frozenset([item])].append((idx,loc))
                    if item not in self.Freq_Items:
                        self.Freq_Items[item]=1
                        self.items.add(frozenset([item]))
                    else:
                        self.Freq_Items[item]+=1
                else:
                    idx+=1
                    if idx>1:
                        self.transactions.append(line)
                    line=[]
            #self.transactions.append(line)
        except IOError as e:
            print("Unable to read dataset file!\n" + e)

    def trans_num(self):
        """Returns the number of transactions in the dataset"""
        return len(self.transactions)

    def items_num(self):
        """Returns the number of different items in the dataset"""
        return len(self.items)

    def get_transaction(self, i):
        """Returns the transaction at index i as an int array"""
        return self.transactions[i]
    def get_items(self):
        return self.items
    def get_freq_items(self,minSup):
        TrNo=len(self.transactions)
        AboveFreq=[i for i in self.Freq_Items if (self.Freq_Items[i]/TrNo)>=minSup]
        return AboveFreq
    
    def get_freq_Vertical_Rep(self,minSup):
        #  it returns the vertical representation of the frequent items
        if minSup!=None:
            TrNo=self.items_num()
            High_freq=self.get_freq_items(minSup)
            Vertical_Rep_high={}
            for it in High_freq:
                Vertical_Rep_high[tuple([it])]=self.Vertical_Rep[tuple([it])]
        return Vertical_Rep_high
    def get_items_in_order(self):
        sequences=[i for i in sorted(self.Freq_Items.items(), key=lambda item: item[1],reverse=True)]
        return sequences
    
class k_selector:
    """Utility class to control k-top elements."""



    def __init__(self, k,mode):
        self.mode=mode # mode could be 'frequent' or 'infrequent'. if it is frequent, it holds top k frequent sequences, otherwise, infrequent 
        self.score_list = list() # a list of top k (or less than k) scores that have been observed so far
        self.score_link = dict() # a dictionary where keys are the scores and the values are the sequences whose scores are equale to the key
        self.k=k # the number of unique scores
    def append(self,score,sequence):
    
        '''
        Input: 
            score e.g, total support or Wracc
            obtained sequence
        Output:
            return True if the sequence could be added to the top-k list sequences
        '''
        ret=False
        if score not in self.score_link:
            if len(self.score_list)<self.k:
                self.score_list.append(score)
                self.score_link[score]=set()
                self.score_link[score].add(sequence)
                ret=True
            else:
                if self.mode=='frequent':
                    minimum=np.min(self.score_list)
                    if score>minimum:
                        self.score_list.pop(self.score_list.index(minimum))
                        del(self.score_link[minimum])
                        self.score_link[score]=set()
                        self.score_list.append(score)
                        self.score_link[score].add(sequence)
                        ret=True
                    if score==minimum:
                        self.score_link[score].add(sequence)
                        ret=True
                elif self.mode=='infrequent':
                    maximum=np.max(self.score_list)
                    if score<maximum:
                        self.score_list.pop(self.score_list.index(maximum))
                        del(self.score_link[maximum])
                        self.score_link[score]=set()
                        self.score_list.append(score)
                        self.score_link[score].add(sequence)
                        ret=True
                    if score==maximum:
                        self.score_link[score].add(sequence)
                        ret=True
        else:
            self.score_link[score].add(sequence)
            ret=True
        return ret
    
def vertical_finding(vr,left_vr,right_item):
    """This function is used to find the sequence using vertical representation (Sparse Lattice suggested in Zaki's paper)"""

    '''
    Input: 
        vertical representation provided by Dataset_Sequence class
        left: vertical representation of the prefix e.g, AB
        right_item= the candidate item added to the prefix e.g, C

    Output:
        res= vertical representation of the prefix + right_item  e.g, ABC
        support= the support of the found sequence
    '''
    right_item=tuple(right_item)
    right=vr.get(right_item,dict())
    res={}
    support=0
    for x in left_vr:
        if x in right:
            break_Flage=False
            for loc_left in left_vr[x]:
                if break_Flage==False:
                    for loc_right in right[x]:
                        if loc_left<loc_right:
                            if x not in res:
                                res[x]=[]
                            res[x].append(loc_right)
                            support+=1
                            break_Flage=True
                            break
                else:
                    break
    return res,support


def topk_spade(data, k,printing=True,mode='frequent',top_frequent=None):
    """ 
    Notice that this top-k spade is a bit different from the one submitted for task1. 
    mode could be 'frequent' or 'infrequent'. if it is frequent, it find top k frequent sequences, otherwise, infrequent 
    Top k SPADE Algorithm- Recursive DFS implementation
    k is the number of top sequences that should be seleceted.
    Notice that, both positive and negative supports for sequences are found in the same call using vertical finding function, thereby speeding up the algorithm.
    
    """

    '''
    Input: 
        data_positive= positive class data provided by Dataset_Sequence class
        data_negative= negative class data provided by Dataset_Sequence class
        k: k is the number of top sequences that should be seleceted.
        printing= if True, print the top k frequent sequences (e.g, [A, B, C] supp_positive supp_negative score)

    Output:
        None 
    '''
    if k==0:
        print('k==0 is not feasible')
        return None
    
    if mode=='infrequent':
        if top_frequent!=None:
            if type(top_frequent)!=dict:
                print('Top_frequent must be a dictionary whose keys are frequent sequences and valuse are the supports')
                return None
        else:
            print('Top_frequent must be a dictionary whose keys are frequent sequences and valuse are the supports')
            return None
    selector=k_selector(k,mode)
    vr=data.get_freq_Vertical_Rep(0) 
    #vr_negative=data_negative.get_freq_Vertical_Rep(0) 
    
    total_trans = data.trans_num() 
    #total_trans_negative = data_negative.trans_num() 
    #total_trans=total_trans_positive+total_trans_negative
    SequenceSet={}
    #items=list(set(data_positive.get_freq_items(0)).union(set(data_negative.get_freq_items(0)))) # find all items observed in positive and negative classes
    items=data.get_freq_items(0)
    ordered_item={}
    # the following block select only top k single items. It is used to prune the search as top k problem benefits from Anti-monotonicity 
    if mode=='frequent':
        for item in items:
            supp=(len(vr.get(tuple([item]),[])))
            #supp_negative=(len(vr_negative.get(tuple([item]),[])))
            selector.append(supp,tuple([item]))
        selected_item=[]
        scores=sorted(selector.score_link.keys(),reverse=True)
        scores=sorted(selector.score_link.keys(),reverse=False)
        for score in scores:
                for sequence in selector.score_link[score]:
                    selected_item.append(sequence)
        selected_item=[i[0] for i in selected_item]
    elif mode=='infrequent':
        selected_item=items
    # end of block 
    for item in selected_item:
        supp=(len(vr.get(tuple([item]),[])))
        #supp_negative=(len(vr_negative.get(tuple([item]),[])))
        if mode=='frequent':
            if supp>0:
                SequenceSet[tuple([item])]=[supp]
                project_vr=vr.get(tuple([item]),dict())
                topk_depthFirstSearch(item, total_trans,vr,SequenceSet,selected_item,project_vr,selector,mode)
        elif mode=='infrequent':
            if supp>0:
                SequenceSet[tuple([item])]=[supp]
                project_vr=vr.get(tuple([item]),dict())
                topk_depthFirstSearch(item, total_trans,vr,SequenceSet,selected_item,project_vr,selector,mode,top_frequent)
            continue

    counter=0
    scores=sorted(selector.score_link.keys(),reverse=True)
    top_k_list={}
    for score in scores:
        for sequence in selector.score_link[score]:
            top_k_list[sequence]=SequenceSet[sequence][0]
            if printing==True:
                out='['+', '.join(list(sequence))+']' 
                print(out,SequenceSet[sequence][0])
            counter+=1
            #print(counter)
    return top_k_list

def topk_depthFirstSearch(item, total_trans,vr,SequenceSet,exploredLocal,projected_vr,selector,mode,top_frequent=None):
    if type(item)!=list:
        item=[item]
    for item2 in exploredLocal:
        item2=[item2]
        if mode=='frequent':
            res,supp=vertical_finding(vr,projected_vr,item2)
            #res_negative,supp_negative=vertical_finding(vr_negative,projected_vr_negative,item2)
            if(supp/total_trans > 0.0):
                sequence=item+item2
                selected=selector.append(supp,tuple(sequence))
                if selected: # if the sequence was added to the top k list, so it is frequent and its supersequences may be top-k frequent sequences as well
                    if tuple(sequence) not in SequenceSet:
                        SequenceSet[tuple(sequence)]=[supp]
                        topk_depthFirstSearch(sequence, total_trans,vr,SequenceSet,exploredLocal,res,selector,mode,top_frequent) # discover supersequences
        elif mode=='infrequent':
            sequence=item+item2
            if tuple(sequence) in top_frequent:
                res,supp=vertical_finding(vr,projected_vr,item2)
                selected=selector.append(supp,tuple(sequence))
                if selected: # if the sequence was added to the top k list, so it is frequent and its supersequences may be top-k frequent sequences as well
                    if tuple(sequence) not in SequenceSet:
                        SequenceSet[tuple(sequence)]=[supp]
                        topk_depthFirstSearch(sequence, total_trans,vr,SequenceSet,exploredLocal,res,selector,mode,top_frequent) # discover supersequences
            
    return SequenceSet
import sys


In [136]:
pos_filepath = 'positive.txt'# filepath to positive class file
neg_filepath = 'negative.txt' # filepath to negative class file
k = 2
# TODO: read the dataset files and call your miner to print the top k itemsets
ds_positive=Dataset_Sequence(pos_filepath)
ds_negative=Dataset_Sequence(neg_filepath)

selected_pos=topk_spade(ds_positive,k,printing=True,mode='frequent')

[C, A] 4
[A, C] 4
[C] 4
[A, A] 4
[A] 4
[A, C, A] 4
[A, A, A] 3
[C, C, A] 3
[A, C, C, A] 3
[A, B] 3
[A, A, C] 3
[A, B, A] 3
[B] 3
[A, C, C] 3
[A, A, C, A] 3
[B, A] 3
[C, C] 3


In [137]:
pos_filepath = 'positive.txt'# filepath to positive class file
neg_filepath = 'negative.txt' # filepath to negative class file
k = 2
# TODO: read the dataset files and call your miner to print the top k itemsets
ds_positive=Dataset_Sequence(pos_filepath)
ds_negative=Dataset_Sequence(neg_filepath)

selected_neg=topk_spade(ds_negative,k,printing=True,mode='infrequent',top_frequent=selected)

[A, C] 1
[A, C, C, A] 1
[A, B, A] 1
[A, C, C] 1
[A, C, A] 1
[A, A, A] 0
[A, A, C, A] 0
[A, A, C] 0


In [134]:
selected_neg

{('A', 'A'): 2,
 ('C', 'C'): 2,
 ('A', 'B'): 2,
 ('C', 'C', 'A'): 2,
 ('A', 'C'): 1,
 ('A', 'C', 'C', 'A'): 1,
 ('A', 'B', 'A'): 1,
 ('A', 'C', 'C'): 1,
 ('A', 'C', 'A'): 1,
 ('A', 'A', 'A'): 0,
 ('A', 'A', 'C', 'A'): 0,
 ('A', 'A', 'C'): 0}

In [151]:
def main():
    pos_filepath = 'positive.txt' # filepath to positive class file
    neg_filepath = 'negative.txt' # filepath to negative class file
    k = 6
    # TODO: read the dataset files and call your miner to print the top k itemsets
    ds_positive=Dataset_Sequence(pos_filepath)
    P=ds_positive.trans_num() #  The number of transactions in positive class
    ds_negative=Dataset_Sequence(neg_filepath)
    N=ds_negative.trans_num() #  The number of transactions in negative class
    total=P+N

    positive_seqs=topk_spade(ds_positive, 6,printing=False,mode='frequent')
    negative_seqs=topk_spade(ds_negative, 6,printing=False,mode='infrequent',top_frequent=positive_seqs)

    wracc_left=(P/total)*(N/total)
    
    Positive_seq_Discoverd=set(positive_seqs.keys())
    Negative_seq_Discoverd=set(negative_seqs.keys())
    To_discovere=Positive_seq_Discoverd.intersection(Negative_seq_Discoverd)
    
    # the following wracc_x is added to compute weighted relative accuracy (Wracc)
    sequences={}
    for sequence in To_discovere:
        wracc_x=wracc_left*((positive_seqs.get(sequence,0)/P)-(negative_seqs.get(sequence,0)/N))
        #sequences[sequence]=[positive_seqs.get(sequence,0),negative_seqs.get(sequence,0),np.round(wracc_x,5)]
        sequences[sequence]=[positive_seqs.get(sequence,0),negative_seqs.get(sequence,0),np.round(wracc_x,5)]

    sequences=sorted(sequences.items(), key=lambda item: item[1][2],reverse=True)
    i=1
    j=0
    prev=sequences[0][1][2]
    for sequence in sequences:
        if sequence[1][2]!=prev:
            i+=1
            prev=sequence[1][2]
        if i<=k:
            out='['+', '.join(list(sequence[0]))+']'
            print(out,sequence[1][0],sequence[1][1],sequence[1][2])
            j+=1
        else:
            break

In [152]:
main()

[A, A, A] 3 0 0.18367
[A, A, C] 3 0 0.18367
[A, A, C, A] 3 0 0.18367
[A, C, A] 4 1 0.16327
[A, C] 4 1 0.16327
[A, C, C, A] 3 1 0.10204
[A, B, A] 3 1 0.10204
[A, C, C] 3 1 0.10204
[A, A] 4 2 0.08163
[B, C, C, A] 1 0 0.06122
[A, C, C, B, A] 1 0 0.06122
[A, C, A, A] 1 0 0.06122
[A, B, B, A, C, A] 1 0 0.06122
[A, C, A, C, A] 1 0 0.06122
[B, A, C, A] 1 0 0.06122
[A, B, C, C, A] 1 0 0.06122
[A, B, B, A, C] 1 0 0.06122
[A, B, A, C] 1 0 0.06122
[A, A, C, C] 1 0 0.06122
[A, B, B, C, A] 1 0 0.06122
[C, C, C] 1 0 0.06122
[A, C, C, A, A] 1 0 0.06122
[B, C, C] 1 0 0.06122
[A, C, A, C] 1 0 0.06122
[A, A, B, A] 1 0 0.06122
[A, B, B, C] 1 0 0.06122
[A, A, B, C, C, A] 1 0 0.06122
[C, A, C, A] 1 0 0.06122
[B, B, A, C, A] 1 0 0.06122
[B, B, A, C] 1 0 0.06122
[A, A, B, C, A] 1 0 0.06122
[A, C, C, A, C, A] 1 0 0.06122
[A, C, C, A, C] 1 0 0.06122
[C, C, A, C, A] 1 0 0.06122
[A, B, C, C] 1 0 0.06122
[A, C, C, B, B] 1 0 0.06122
[A, C, C, B, A, B] 1 0 0.06122
[A, B, A, C, A] 1 0 0.06122
[A, C, C, C, A] 1 0 0.0